# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

<b>Условия задачи: </b>
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

<b>Инструкция по выполнению проекта</b><br>

<b>Загрузите и подготовьте данные. Поясните порядок действий.</b>
-  Обучите и проверьте модель для каждого региона:
-  Разбейте данные на обучающую и валидационную выборки в соотношении 75:25.
-  Обучите модель и сделайте предсказания на валидационной выборке.
-  Сохраните предсказания и правильные ответы на валидационной выборке.
-  Напечатайте на экране средний запас предсказанного сырья и RMSE модели.
-  Проанализируйте результаты.<br><br>
<b>Подготовьтесь к расчёту прибыли:</b>
-  Все ключевые значения для расчётов сохраните в отдельных переменных.
-  Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины. Сравните полученный объём сырья со средним запасом в каждом регионе.
-  Напишите выводы по этапу подготовки расчёта прибыли.<br><br>
<b>Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:</b>
-  Выберите скважины с максимальными значениями предсказаний.
-  Просуммируйте целевое значение объёма сырья, соответствующее этим предсказаниям.
-  Рассчитайте прибыль для полученного объёма сырья.<br><br>
<b>Посчитайте риски и прибыль для каждого региона:</b>
-  Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли.
-  Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков. Убыток — это отрицательная прибыль.
-  Напишите выводы: предложите регион для разработки скважин и обоснуйте выбор.

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

## Загрузка и подготовка данных

In [2]:
try:
    df_1 = pd.read_csv('/datasets/geo_data_0.csv')
    df_2 = pd.read_csv('/datasets/geo_data_1.csv')
    df_3 = pd.read_csv('/datasets/geo_data_2.csv')
except:
    df_1 = pd.read_csv('geo_data_0.csv')
    df_2 = pd.read_csv('geo_data_1.csv')
    df_3 = pd.read_csv('geo_data_2.csv')

In [3]:
dfs = [df_1, df_2, df_3]

In [4]:
for i in dfs:
    print(i.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column  

In [5]:
for i in dfs:
    print(i.head())

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647
      id         f0         f1        f2     product
0  kBEdx -15.001348  -8.276000 -0.005876    3.179103
1  62mP7  14.272088  -3.475083  0.999183   26.953261
2  vyE1P   6.263187  -5.948386  5.001160  134.766305
3  KcrkZ -13.081196 -11.506057  4.999415  137.945408
4  AHL4O  12.702195  -8.147433  5.004363  134.766305
      id        f0        f1        f2     product
0  fwXo0 -1.146987  0.963328 -0.828965   27.758673
1  WJtFt  0.262778  0.269839 -2.530187   56.069697
2  ovLUW  0.194587  0.289035 -5.586433   62.871910
3  q6cA6  2.236060 -0.553760  0.930038  114.572842
4  WPMUX -0.515993  1.716266  5.899011  149.600746


In [6]:
for i in dfs:
    print(i.describe())

                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021      26.953261
50%       

Данные загружены корректно. Далее имеет смысл колонку id сделать индексом.

In [7]:
def prepare_df(data):
    """ присваивает значения колонки id индексу """
    data.index = data['id']
    data = data.drop('id', axis = 1)
    return data
 
regions_data = []
for i in dfs:
    regions_data.append(prepare_df(i)) 

In [8]:
def feature_target(data):
    """разбивает данные в датафрэйме на признаки и цель"""
    feature = data.drop('product', axis = 1)
    target = data['product']
    return feature, target

<b> Вывод </b> <br>
Данные загружены корректно.  Колонка id преобразована в значения индекса.

## Обучение и проверка модели

In [9]:
model = LinearRegression()
oil_reserves = {}
RMSE = {}
target_valid_regions = {}
num_region = 1
for i in regions_data:
    feature, target = feature_target(i)
    feature_train, feature_valid, target_train, target_valid = train_test_split(
    feature, target, test_size = 0.25, random_state = 12345)
    reg = model.fit(feature_train, target_train)
    predictions_valid = reg.predict(feature_valid)
    rms = (mean_squared_error(target_valid, predictions_valid))**0.5
    oil_reserves.setdefault(str(num_region),predictions_valid)
    RMSE.setdefault(str(num_region), rms)
    target_valid_regions.setdefault(str(num_region), target_valid)
    num_region += 1

In [10]:
for i in oil_reserves.keys():
    print(f'среднее количество запасов нефти в скважине в регионе {i} равно: {oil_reserves[i].mean():.2f} тыс. баррелей\
    \nRMSE модели для региона {i} равно: {RMSE[i]}')

среднее количество запасов нефти в скважине в регионе 1 равно: 92.59 тыс. баррелей    
RMSE модели для региона 1 равно: 37.5794217150813
среднее количество запасов нефти в скважине в регионе 2 равно: 68.73 тыс. баррелей    
RMSE модели для региона 2 равно: 0.8930992867756168
среднее количество запасов нефти в скважине в регионе 3 равно: 94.97 тыс. баррелей    
RMSE модели для региона 3 равно: 40.02970873393434


<b>Вывод:</b> <br> регионы 1 и 3, согласно предсказанию модели, имеют большие запасы чем регион 2, однако точность предсказания у модели для региона 2 значительно выше.  

## Подготовка к расчёту прибыли

In [11]:
EXPLORATION_COSTS = 10**10
EXPLORATION_WELLS_NUMBER = 500
WORKING_WELLS_NUMBER = 200
BARREL_COST = 450

In [12]:
EXPLORATION_COSTS_ONE_WELLS = EXPLORATION_COSTS/EXPLORATION_WELLS_NUMBER
EXPLORATION_COSTS_ONE_WELLS

20000000.0

In [13]:
PAYBACK_TRESHHOLD_ONE_WORKING_WELL = EXPLORATION_COSTS/WORKING_WELLS_NUMBER
PAYBACK_TRESHHOLD_ONE_WORKING_WELL

50000000.0

In [14]:
ENOUGHT_VOLUME = PAYBACK_TRESHHOLD_ONE_WORKING_WELL/(BARREL_COST*1000)
print(f'необходимый запас нефти в скважине для выхода на прибыль должен быть больше: {ENOUGHT_VOLUME:.2f} тыс. баррелей')

необходимый запас нефти в скважине для выхода на прибыль должен быть больше: 111.11 тыс. баррелей


In [15]:
def revenue(predictions, target, WORKING_WELLS_NUMBER):
    """функция определяет прибыль из разработанных скважин. 
    Функция  на вход получает предсказанные значения, целевые показатели  сырья в скважинах 
    и количество скважин для бурения.
    Далее предсказанные значения сортирует по убыванию, берет 200 первых скважин, 
    и согласно их индексам отбирает значения целевого признака, далее суммируется количество сырья 
    в 'целевых' скважинах, умножается на 1000 и из полученного значения вычитается стоимость исследований и разработки"""
    probs_sorted = predictions.sort_values(by = 0 , ascending=False)
    target = target[probs_sorted[:WORKING_WELLS_NUMBER].index]
    selected = BARREL_COST * target.sum()
    return 1000 * selected - EXPLORATION_COSTS
    

<b>Вывод:</b>
    
Средние значения запасов нефти в скажинах по регионам значительно меньше чем требуется для получения прибыли, поэтому крайне важно максимально точно определить локацию для бурения новой скважины.

## Расчёт прибыли и рисков 

In [16]:
def risk_value(data, number_samples):
    """определяет долю риска получения убытков.
    функция получает на вход значения прибыли по бутстреп выборкам находит отрицательные значения,
    суммирует их и делит на количество выборок"""
    n = 0
    for i in data:
        if i < 0:
            n += 1        
    return n/number_samples
    

In [17]:
def prediction_income_loss(predictions, target, region_name):
    """функция определяет значения прибыли и риска убытков по регионам""" 
    alpha = 0.05
    incomes = []
    losses = []
    state = np.random.RandomState(12345)
    number_samples = 1000
    for _ in range(number_samples):
        data = pd.DataFrame(predictions[region_name])
        target_subsample = data.sample(n=EXPLORATION_WELLS_NUMBER, replace=True, random_state=state)
        incomes.append(revenue(target_subsample, target[region_name], WORKING_WELLS_NUMBER))
    conf_int_left = pd.Series(incomes).quantile(alpha/2)
    conf_int_right = pd.Series(incomes).quantile(1 - alpha/2)
    loss = (pd.Series(incomes) < 0).mean()
    #loss = risk_value(incomes, number_samples)
    income = np.mean(incomes)
    return region_name, income, loss, conf_int_left, conf_int_right
    
        

In [18]:
regions_number = 0
max_revenue = 0
loss_rate = 0
for i in oil_reserves.keys():
    region_name, income, loss, conf_int_left, conf_int_right =  prediction_income_loss(oil_reserves, target_valid_regions, i)
    print(f'Средняя прибыль в регионе {region_name} составляет {(income/10**6):.2f} (млн. Р)')
    print(f'Вероятность убытка в регионе {region_name} составляет {loss:%}')
    print(f'95% доверительный интервал: {(conf_int_left/10**6):.2f} : {(conf_int_right/10**6):.2f} (млн. Р)')
    #print(f'доверительный интервал для региона {region_name} - {conf_int_left} : {conf_int_right} ')
    if income > max_revenue:
        max_revenue = income
        regions_number = i
        loss_rate = loss
    
print(f'Скважину следует бурить в регионе {regions_number}, т.к. именно этот регион дает наибольшую среднюю прибыль равную {max_revenue/10**6:.2f} млн. руб., при риске получения убытка равном {loss_rate:.2%}')
    

Средняя прибыль в регионе 1 составляет 396.16 (млн. Р)
Вероятность убытка в регионе 1 составляет 6.900000%
95% доверительный интервал: -111.22 : 909.77 (млн. Р)
Средняя прибыль в регионе 2 составляет 456.05 (млн. Р)
Вероятность убытка в регионе 2 составляет 1.500000%
95% доверительный интервал: 33.82 : 852.29 (млн. Р)
Средняя прибыль в регионе 3 составляет 404.40 (млн. Р)
Вероятность убытка в регионе 3 составляет 7.600000%
95% доверительный интервал: -163.35 : 950.36 (млн. Р)
Скважину следует бурить в регионе 2, т.к. именно этот регион дает наибольшую среднюю прибыль равную 456.05 млн. руб., при риске получения убытка равном 1.50%


<b>Вывод:</b> <br>
Согласно полученным расчетам, скважину следует бурить в регионе 2, т.к., именно здесь, достигается наибольшая прибыль при наименьшем риске получения убытка. 


## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован